In [122]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import re as re

# Chargement des données

In [123]:
dfm=pd.read_csv('paris-march.csv')
dfj= pd.read_csv('paris-june.csv')
dfd = pd.read_csv('paris-december.csv')
df = pd.concat([dfm,dfj,dfd])
df.head(20)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3109,https://www.airbnb.com/rooms/3109,20240316023852,2024-03-19,city scrape,zen and calm,Lovely Appartment with one bedroom with a Quee...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/miso/Hosting-...,3631,...,5.00,5.00,5.00,7511409139079,f,1,1,0,0,0.05
1,165409,https://www.airbnb.com/rooms/165409,20240316023852,2024-03-19,city scrape,studio charme Haut Marais PARIS,You'll love this typical parisian flat.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,788732,...,4.89,4.95,4.74,7510309597559,f,1,1,0,0,0.69
2,5396,https://www.airbnb.com/rooms/5396,20240316023852,2024-03-18,city scrape,Your perfect Paris studio on Île Saint-Louis,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.84,4.96,4.58,7510402838018,f,2,1,1,0,2.19
3,166370,https://www.airbnb.com/rooms/166370,20240316023852,2024-03-16,city scrape,Big appartment with view 4 people,"2 Bedrooms apartment for up to 4 people, with ...",The avenue is located nearby the animated dist...,https://a0.muscache.com/pictures/74224563/6c9e...,793032,...,4.79,4.65,4.48,7511007974397,f,1,1,0,0,0.51
4,39948,https://www.airbnb.com/rooms/39948,20240316023852,2024-03-19,previous scrape,COSY INDEPENDENT ROOM IN CENTRAL PARIS,Large newly-renovated guest room in central Pa...,"Located in the heart of Paris, The Montorgueil...",https://a0.muscache.com/pictures/f898fb0b-e72e...,171483,...,4.97,4.94,4.82,7510200734360,f,1,0,1,0,0.99
5,40143,https://www.airbnb.com/rooms/40143,20240316023852,2024-03-19,previous scrape,Stylish & Design 1BR flat- Voltaire Bastille,I have bought and redesigned completely my fla...,"The flat is located in Voltaire (11th), in a q...",https://a0.muscache.com/pictures/1659858e-41a7...,172322,...,4.88,4.88,5.00,NaN,f,1,1,0,0,0.07
6,7397,https://www.airbnb.com/rooms/7397,20240316023852,2024-03-18,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !",NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,4.88,4.93,4.73,7510400829623,f,6,6,0,0,2.21
7,40899,https://www.airbnb.com/rooms/40899,20240316023852,2024-03-18,city scrape,Lovely flat Canal Saint Martin Long stays,"Very bright and airy, with a lovely view over ...","La Place Sainte-Marthe, le canal Saint-Martin,...",https://a0.muscache.com/pictures/69330435/8e1a...,177130,...,4.93,4.79,4.72,7511000493125,f,1,1,0,0,0.92
8,41106,https://www.airbnb.com/rooms/41106,20240316023852,2024-03-16,city scrape,"Comfort & style (?) in le Marais, what else?",Bonjour :-)<br /><br />This apartment of 50 m2...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,177387,...,4.94,4.96,4.69,7510400114395,f,1,1,0,0,0.67
9,167998,https://www.airbnb.com/rooms/167998,20240316023852,2024-03-18,city scrape,"Cosy luxe, Central Paris.","A local neighbourhood filled with history, res...",NaN,https://a0.muscache.com/pictures/066c69ab-60ef...,469865,...,4.86,4.98,4.53,7510103301622,f,2,2,0,0,1.19


# Pré-traitement des données

In [127]:
#on selectionnne les champs dont nouos avions besoin
column_index = ['host_is_superhost','host_location', 'accommodates','beds','bedrooms','number_of_reviews','review_scores_rating','host_listings_count','availability_30','minimum_nights', 'bathrooms', 'property_type', 'room_type', 'amenities', 'price']

df = df[column_index]
df.head(20)


,host_is_superhost,host_location,accommodates,beds,bedrooms,number_of_reviews,review_scores_rating,host_listings_count,availability_30,minimum_nights,bathrooms,property_type,room_type,amenities,price
0,f,"Paris, France",2,1.0,1.0,4,5.00,1.0,29,2,1.0,Entire rental unit,Entire home/apt,"[""Cooking basics"", ""Luggage dropoff allowed"", ...",$250.00
1,t,"Paris, France",2,1.0,0.0,106,4.68,1.0,15,3,1.0,Entire rental unit,Entire home/apt,"[""Lockbox"", ""Wifi"", ""Kitchen"", ""Refrigerator"",...",$140.00
2,f,"Paris, France",2,1.0,0.0,393,4.60,2.0,0,1,1.0,Entire rental unit,Entire home/apt,"[""Cooking basics"", ""Long term stays allowed"", ...",$107.00
3,f,"Paris, France",4,1.0,1.0,78,4.56,1.0,29,2,1.0,Entire rental unit,Entire home/apt,"[""Shampoo"", ""Kitchen"", ""Washer"", ""Cooking basi...",$200.00
4,t,"Paris, France",2,NaN,NaN,126,4.86,1.0,0,5,NaN,Private room in guest suite,Private room,"[""Extra pillows and blankets"", ""Wifi"", ""Privat...",NaN
5,f,"Paris, France",4,NaN,1.0,11,5.00,2.0,0,365,NaN,Entire rental unit,Entire home/apt,"[""Shampoo"", ""Carbon monoxide alarm"", ""Smoke al...",NaN
6,t,"Paris, France",4,2.0,2.0,348,4.73,6.0,0,10,1.0,Entire rental unit,Entire home/apt,"[""Cooking basics"", ""Wifi"", ""Dedicated workspac...",$134.00
7,f,"Paris, France",2,1.0,1.0,100,4.67,1.0,0,20,1.0,Entire rental unit,Entire home/apt,"[""Cooking basics"", ""Long term stays allowed"", ...",$50.00
8,t,"Paris, France",4,2.0,1.0,111,4.76,1.0,25,5,1.0,Entire rental unit,Entire home/apt,"[""Shampoo"", ""Freezer"", ""Kitchen"", ""Extra pillo...",$265.00
9,f,"Paris, France",4,3.0,2.0,177,4.72,2.0,9,1,1.5,Entire rental unit,Entire home/apt,"[""Cooking basics"", ""Pets allowed"", ""Wifi"", ""Po...",$443.00


# Verification des duplications

In [ ]:
print(df.duplicated().sum())
if df.duplicated().sum()>0:
    df.drop_duplicates()

# Supression des champs vides et conversion

In [ ]:
#Remplacer toutes les valeurs numériques par la mediane
# Colonnes numériques
numerical_columns = ['accommodates', 'bedrooms', 'bathrooms','beds']

# Créer un SimpleImputer avec la stratégie de remplacement par la médiane
imputer = SimpleImputer(strategy='median')

# Remplacer les valeurs nulles dans les colonnes numériques par leur médiane
df[numerical_columns] = imputer.fit_transform(df[numerical_columns])

#Remplacer toutes les valeurs numériques par la moyenne
# Colonnes numériques
numerical_columns = ['number_of_reviews', 'review_scores_rating','price']

# Créer un SimpleImputer avec la stratégie de remplacement par la médiane
imputer = SimpleImputer(strategy='median')

# Remplacer les valeurs nulles dans les colonnes numériques par leur médiane
df[numerical_columns] = imputer.fit_transform(df[numerical_columns])
# Remplacer les valeurs 't' par 1, les valeurs 'f' par 0, et les valeurs manquantes par 0
df['host_is_superhost'] = np.where(df['host_is_superhost'] == 't', 1, df['host_is_superhost'])
df['host_is_superhost'] = np.where(df['host_is_superhost'] == 'f', 0, df['host_is_superhost'])
df['host_is_superhost'] = df['host_is_superhost'].fillna(0).infer_objects(copy=False).astype(int)
